# Deepfake Face Swap: Gambar ke Video File di Google Colab

Notebook ini memandu Anda untuk melakukan face swap dari gambar sumber ke wajah yang terdeteksi di setiap frame dari file video target menggunakan `insightface`.

## 1. Setup Environment

Instalasi library yang dibutuhkan.

In [ ]:
# CRUCIAL: Instalasi library utama
!pip install -q insightface==0.7.3 onnxruntime-gpu opencv-python-headless numpy tqdm

**PENTING:** Setelah menjalankan sel instalasi di atas, Anda **HARUS** me-restart runtime agar library yang baru diinstal dapat dikenali dengan benar. 
Caranya: Pilih menu `Runtime` -> `Restart runtime` (atau `Ctrl+M .`). Setelah itu, lanjutkan menjalankan sel berikutnya.

## 2. Import Libraries dan Inisialisasi Model Utama

Mengimpor library dan menginisialisasi model `FaceAnalysis` serta model `swapper`. Model swapper akan diunduh dari URL Hugging Face.

In [ ]:
import cv2
import numpy as np
import insightface # Library utama
from insightface.app import FaceAnalysis
import matplotlib.pyplot as plt 
import os
import subprocess # Untuk wget
import traceback # Untuk debugging
from google.colab import files # Untuk upload/download file Colab
from tqdm.auto import tqdm # Progress bar

# CRUCIAL: URL untuk model swapper dari Hugging Face (menggunakan link /resolve/ untuk unduhan langsung)
URL_MODEL_SWAPPER = "https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx"

providers = [] # Akan diisi berdasarkan ketersediaan GPU
try:
    import onnxruntime
    print(f"ONNXRuntime versi: {onnxruntime.__version__}")
    print(f"ONNXRuntime provider tersedia: {onnxruntime.get_available_providers()}")
    if 'CUDAExecutionProvider' in onnxruntime.get_available_providers():
        providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] 
        print("CUDAExecutionProvider (GPU) akan digunakan.")
    else:
        providers = ['CPUExecutionProvider']
        print("CUDAExecutionProvider tidak ditemukan, menggunakan CPUExecutionProvider.")
except ImportError:
    print("ONNXRuntime tidak terinstal dengan benar.")
    providers = ['CPUExecutionProvider']

print(f"Provider ONNXRuntime yang akan digunakan: {providers}")
print(f"Insightface versi: {insightface.__version__}")

# CRUCIAL: Inisialisasi FaceAnalysis untuk deteksi wajah
print("\nInisialisasi FaceAnalysis (model buffalo_l)...")
app = FaceAnalysis(name='buffalo_l', root='~/.insightface', providers=providers)
app.prepare(ctx_id=0, det_size=(640, 640)) 
print("Model FaceAnalysis (buffalo_l) siap.")

# CRUCIAL: Inisialisasi model untuk face swapping
print("\nInisialisasi model swapper...")
swapper = None
try:
    models_dir = os.path.expanduser('~/.insightface/models')
    os.makedirs(models_dir, exist_ok=True)
    model_filename = 'inswapper_128.onnx'
    model_path = os.path.join(models_dir, model_filename)
    print(f"Path target untuk model swapper: {model_path}")

    if not URL_MODEL_SWAPPER or URL_MODEL_SWAPPER == "MASUKKAN_URL_LANGSUNG_KE_FILE_INSWAPPER_128_ONNX_DI_SINI":
        print("URL_MODEL_SWAPPER tidak valid. Tidak bisa mengunduh model.")
        if not os.path.exists(model_path):
             raise FileNotFoundError(f"Model swapper tidak ditemukan di {model_path} dan URL tidak disediakan.")
    else:
        print(f"URL untuk mengunduh model swapper: {URL_MODEL_SWAPPER}")
        if os.path.exists(model_path):
            file_size_mb = os.path.getsize(model_path) / (1024*1024)
            print(f"Model swapper lama ditemukan ({file_size_mb:.2f} MB). Menghapus sebelum mengunduh yang baru...")
            os.remove(model_path)
        
        print(f"Mengunduh model swapper dari URL ke '{model_path}'...")
        subprocess.run(['wget', '-nv', '--show-progress', '-O', model_path, URL_MODEL_SWAPPER], check=True)
        print("Model swapper berhasil diunduh.")
        
        if os.path.exists(model_path):
            file_size_mb = os.path.getsize(model_path) / (1024*1024)
            print(f"Ukuran file model yang diunduh: {file_size_mb:.2f} MB.")
            if file_size_mb < 40: 
                print("PERINGATAN: Ukuran file model yang diunduh sangat kecil!")
        else:
            raise FileNotFoundError("Model tidak ditemukan setelah proses unduh.")

    if os.path.exists(model_path):
        print(f"\nMencoba memuat model swapper dari: {model_path}")
        swapper = insightface.model_zoo.get_model(model_path, providers=providers)
        print("Model swapper berhasil dimuat.")
    else:
        print(f"File model swapper tidak ditemukan di '{model_path}'.")
        raise FileNotFoundError(f"Model swapper tidak ada di {model_path}.")

except Exception as e_model_init:
    print(f"\n--- ERROR SAAT INISIALISASI MODEL SWAPPER --- ")
    traceback.print_exc()
    swapper = None 

if app and swapper:
    print("\nModel FaceAnalysis (app) dan Swapper (swapper) berhasil diinisialisasi.")
else:
    print("\nGAGAL menginisialisasi satu atau lebih model penting.")

## 3. Unggah Gambar Sumber (Wajah Asli)

Unggah gambar yang berisi wajah yang ingin Anda **gunakan** sebagai wajah pengganti. Fitur dari wajah ini akan diekstrak.

In [ ]:
source_image_cv_video = None 
source_face_video = None    # CRUCIAL: Variabel untuk menyimpan fitur wajah sumber untuk video

def upload_and_extract_source_face_for_video(face_analysis_app_video):
    print("Silakan unggah gambar untuk wajah sumber (akan digunakan untuk video):")
    uploaded_files_video = files.upload()
    if not uploaded_files_video: return None, None

    img_name_video = list(uploaded_files_video.keys())[0]
    img_bytes_video = uploaded_files_video[img_name_video]
    nparr_video = np.frombuffer(img_bytes_video, np.uint8)
    img_cv_video = cv2.imdecode(nparr_video, cv2.IMREAD_COLOR)

    if img_cv_video is None: print(f"Gagal membaca '{img_name_video}'."); return None, None
    print(f"Gambar sumber '{img_name_video}' diunggah.")

    print("Mendeteksi wajah pada gambar sumber untuk video...")
    faces_in_source_video = face_analysis_app_video.get(img_cv_video)
    if not faces_in_source_video: print("ERROR: Tidak ada wajah terdeteksi di gambar sumber!"); return img_cv_video, None
    
    main_source_face_video = faces_in_source_video[0]
    print(f"Ditemukan {len(faces_in_source_video)} wajah. Menggunakan wajah pertama sebagai sumber untuk video.")
    return img_cv_video, main_source_face_video

if 'app' in locals() and app is not None: # Menggunakan 'app' yang sama
    source_image_cv_video, source_face_video = upload_and_extract_source_face_for_video(app)
    if source_face_video is not None:
        print("Fitur wajah sumber ('source_face_video') untuk video siap.")
    else:
        print("Gagal mendapatkan 'source_face_video'.")
else:
    print("Model 'app' belum siap.")

## 4. Proses Face Swap pada File Video Target

Unggah file video target. Wajah dari `source_face_video` akan ditempelkan ke setiap frame video ini.

In [ ]:
# CRUCIAL: Fungsi untuk memproses file video
def process_video_file_face_swap(source_face_features, face_analysis_app, face_swapping_model):
    if source_face_features is None: print("ERROR: 'source_face_features' tidak ada."); return
    if face_analysis_app is None or face_swapping_model is None: print("ERROR: Model 'app' atau 'swapper' belum siap."); return

    print("Unggah file video target:")
    uploaded_video = files.upload()
    if not uploaded_video: print("Tidak ada video diunggah."); return

    input_video_name = list(uploaded_video.keys())[0]
    input_video_path = input_video_name 
    print(f"\nVideo untuk diproses: '{input_video_path}'")

    video_capture = cv2.VideoCapture(input_video_path)
    if not video_capture.isOpened(): print(f"Error buka video '{input_video_path}'."); return

    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Detail: {frame_width}x{frame_height} @ {fps:.2f} FPS, Total: {total_frames} frames")

    if total_frames == 0:
        print("Peringatan: Total frame 0.")
        ret_test, _ = video_capture.read()
        if not ret_test: print("Gagal baca frame pertama."); video_capture.release(); return
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0) 

    base_output_name = os.path.splitext(input_video_name)[0]
    output_video_filename = f"{base_output_name}_swapped_video.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    video_writer = cv2.VideoWriter(output_video_filename, fourcc, fps, (frame_width, frame_height))
    print(f"Output video: '{output_video_filename}'")
    print("Memproses video file (bisa lama)... ")

    # CRUCIAL: Loop pemrosesan setiap frame video
    for _ in tqdm(range(total_frames), desc="Memproses Video File", unit="frame"):
        ret, current_frame = video_capture.read()
        if not ret: print("\nPeringatan: Frame tidak terbaca."); break

        processed_frame_output = current_frame.copy()
        try:
            target_faces_in_frame = face_analysis_app.get(current_frame)
            if target_faces_in_frame:
                first_target_face = target_faces_in_frame[0] 
                processed_frame_output = face_swapping_model.get(processed_frame_output, first_target_face, source_face_features, paste_back=True)
            video_writer.write(processed_frame_output) 
        except Exception as e_frame_proc_file:
            print(f"\nError proses frame video: {e_frame_proc_file}")
            video_writer.write(current_frame) 

    video_capture.release()
    video_writer.release()
    print(f"\n--- Pemrosesan Video File Selesai ---")
    print(f"Video hasil disimpan: '{output_video_filename}'")
    print("Unduh dari panel file Colab atau gunakan 'files.download()'.")
    print("PERHATIAN: Video hasil TANPA AUDIO.")

# --- Pemanggilan Fungsi --- 
if 'source_face_video' in locals() and source_face_video is not None and \
   'app' in locals() and app is not None and \
   'swapper' in locals() and swapper is not None:
    
    print("Model & wajah sumber siap. Untuk proses video file, hapus '#' di bawah & jalankan lagi sel ini.")
    # CRUCIAL: Hilangkan komentar di bawah ini untuk menjalankan pemrosesan video file
    # process_video_file_face_swap(source_face_video, app, swapper)
else:
    print("--- PERINGATAN --- ")
    if 'source_face_video' not in locals() or source_face_video is None: print("- 'source_face_video' BELUM SIAP.")
    if 'app' not in locals() or app is None: print("- Model 'app' BELUM SIAP.")
    if 'swapper' not in locals() or swapper is None: print("- Model 'swapper' BELUM SIAP.")

## 5. Penjelasan Tambahan

- **Audio**: Video hasil tidak akan menyertakan audio asli. Anda memerlukan alat tambahan seperti `ffmpeg` untuk menggabungkan audio kembali jika diperlukan.
- **Waktu Proses**: Pemrosesan video bisa memakan waktu lama tergantung durasi, resolusi, dan FPS video.
- **Kualitas Hasil**: Sangat bergantung pada kualitas input dan kemiripan wajah.
- **Beberapa Wajah**: Kode menukar wajah pertama yang terdeteksi di setiap frame. Untuk skenario lebih kompleks, modifikasi kode diperlukan.